# Lineup Optimization for Maximizing GD per Stint

This notebook optimizes player lineups for teams (e.g., Canada) to maximize goal differential (GD) per stint, constrained by the total player rating sum ≤ 8 per stint.

In [9]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict

# Define data directory
DATA_DIR = 'data/'

## Section 1: Load and Prepare Data

In [10]:
# Load player data
player_data = pd.read_csv(DATA_DIR + 'player_data.csv')

# Extract normalized stint_gd from player_data
stint_gd = {}
for _, row in player_data.iterrows():
    player = row['player']
    stint_gd[player] = {}
    for i in range(1, 17):
        col = f'stint{i}_gd'
        if col in row.index and not pd.isna(row[col]):
            stint_gd[player][i] = row[col]

# Extract WOWY data
wowy_dict = dict(zip(player_data['player'], player_data['wowy']))

print("Normalized stint_gd and WOWY loaded from player_data")


Normalized stint_gd and WOWY loaded from player_data


## Section 2: Filter for Team (Canada)

In [11]:
# Filter for Canada
team = 'Canada'
team_players = player_data[player_data['team'] == team]

print(f"Team: {team}, Players: {len(team_players)}")

Team: Canada, Players: 12


## Section 3: Optimize Lineups per Stint


In [12]:
# MILP optimization with Gurobi
import gurobipy as gp
from gurobipy import GRB


def optimize_lineup_milp(stint_num, players_df, stint_gd_dict, wowy_dict, rating_limit=8, wowy_factor=1.0):
    # Filter players with stint data
    valid_df = players_df[players_df['player'].apply(lambda p: stint_num in stint_gd_dict.get(p, {}))]
    if valid_df.empty:
        return None, None, None

    players = valid_df['player'].tolist()
    ratings = valid_df.set_index('player')['rating'].to_dict()
    scores = {p: stint_gd_dict[p][stint_num] + wowy_dict.get(p, 0) * wowy_factor for p in players}

    model = gp.Model()
    model.Params.OutputFlag = 0
    x = {p: model.addVar(vtype=GRB.BINARY, name=p) for p in players}

    # Objective
    model.setObjective(gp.quicksum(scores[p] * x[p] for p in players), GRB.MAXIMIZE)
    # Constraints
    model.addConstr(gp.quicksum(x[p] for p in players) == 4)
    model.addConstr(gp.quicksum(ratings[p] * x[p] for p in players) <= rating_limit)

    model.optimize()
    if model.Status != GRB.OPTIMAL:
        return None, None, None

    sel = [p for p in players if x[p].X > 0.5]
    best_score = model.ObjVal
    rating_sum = sum(ratings[p] for p in sel)
    return sel, best_score, rating_sum



In [13]:
# Function to optimize lineup for a stint
def optimize_lineup(stint_num, players_df, stint_gd_dict, wowy_dict, rating_limit=8, wowy_factor=1.0):
    players = players_df['player'].tolist()
    ratings = players_df.set_index('player')['rating'].to_dict()
    
    # Get players with GD for this stint
    valid_players = [p for p in players if stint_num in stint_gd_dict.get(p, {})]
    if len(valid_players) < 4:
        return None, None, None
    
    best_score = float('-inf')
    best_lineup = None
    best_rating_sum = None
    
    # Brute force: try all combinations of 4 players
    for combo in combinations(valid_players, 4):
        rating_sum = sum(ratings[p] for p in combo)
        if rating_sum <= rating_limit:
            # Score = sum of (GD + WOWY * factor)
            score = sum(stint_gd_dict[p][stint_num] + wowy_dict.get(p, 0) * wowy_factor for p in combo)
            if score > best_score:
                best_score = score
                best_lineup = combo
                best_rating_sum = rating_sum
    
    return best_lineup, best_score, best_rating_sum

# Get max stint number
max_stint = max(max(stint_gd[p].keys()) for p in stint_gd if stint_gd[p])

# Optimize for each stint (MILP if available, else brute force)
optimized_lineups = {}
for stint in range(1, max_stint + 1):
    lineup, score, rating = optimize_lineup_milp(stint, team_players, stint_gd, wowy_dict)
    if lineup:
        optimized_lineups[stint] = {'lineup': lineup, 'score': score, 'rating_sum': rating}
    else:
        optimized_lineups[stint] = None

print("Optimization completed with Gurobi MILP")


Optimization completed with Gurobi MILP


## Section 4: Display Results


In [14]:
# Display results
for stint, data in optimized_lineups.items():
    if data:
        print(f"Stint {stint}: Score = {data['score']:.2f}, Rating Sum = {data['rating_sum']:.2f}")
        print(f"  Lineup: {', '.join(data['lineup'])}")
    else:
        print(f"Stint {stint}: No valid lineup found")
    print()


Stint 1: Score = 0.34, Rating Sum = 8.00
  Lineup: Canada_p3, Canada_p6, Canada_p9, Canada_p11

Stint 2: Score = 1.00, Rating Sum = 8.00
  Lineup: Canada_p2, Canada_p5, Canada_p6, Canada_p7

Stint 3: Score = 1.44, Rating Sum = 8.00
  Lineup: Canada_p2, Canada_p3, Canada_p5, Canada_p9

Stint 4: Score = 0.55, Rating Sum = 8.00
  Lineup: Canada_p3, Canada_p6, Canada_p9, Canada_p11

Stint 5: Score = 1.92, Rating Sum = 5.50
  Lineup: Canada_p6, Canada_p9, Canada_p10, Canada_p12

Stint 6: Score = 2.30, Rating Sum = 7.00
  Lineup: Canada_p2, Canada_p8, Canada_p9, Canada_p10

Stint 7: Score = 1.67, Rating Sum = 4.50
  Lineup: Canada_p2, Canada_p6, Canada_p10, Canada_p11

Stint 8: Score = 2.47, Rating Sum = 8.00
  Lineup: Canada_p1, Canada_p6, Canada_p7, Canada_p12

Stint 9: Score = 0.83, Rating Sum = 8.00
  Lineup: Canada_p1, Canada_p9, Canada_p11, Canada_p12

Stint 10: Score = 1.21, Rating Sum = 6.50
  Lineup: Canada_p2, Canada_p6, Canada_p7, Canada_p11

Stint 11: Score = 1.06, Rating Sum = 7

## Section 5: Compare optimized vs actual Canada stints


In [15]:
# Compare optimized lineups vs actual Canada stint outcomes
from collections import defaultdict
import statistics, math

stint_df = pd.read_csv(DATA_DIR + 'stint_data.csv')

# Recompute stint_number per game
stint_df = stint_df.copy()
stint_df['original_order'] = range(len(stint_df))
stint_df = stint_df.sort_values(['game_id', 'original_order'])
stint_df['stint_number'] = stint_df.groupby('game_id').cumcount() + 1

# Team strength from standings
game_results = stint_df.groupby('game_id').agg({
    'h_goals': 'sum',
    'a_goals': 'sum',
    'h_team': 'first',
    'a_team': 'first'
}).reset_index()
team_stats = defaultdict(lambda: {'wins': 0, 'draws': 0, 'losses': 0, 'points': 0, 'gd': 0})
for _, row in game_results.iterrows():
    h_team, a_team = row['h_team'], row['a_team']
    gd = row['h_goals'] - row['a_goals']
    if gd > 0:
        team_stats[h_team]['wins'] += 1; team_stats[h_team]['points'] += 3; team_stats[a_team]['losses'] += 1
    elif gd < 0:
        team_stats[a_team]['wins'] += 1; team_stats[a_team]['points'] += 3; team_stats[h_team]['losses'] += 1
    else:
        team_stats[h_team]['draws'] += 1; team_stats[h_team]['points'] += 1; team_stats[a_team]['draws'] += 1; team_stats[a_team]['points'] += 1
    team_stats[h_team]['gd'] += gd; team_stats[a_team]['gd'] -= gd
team_df = pd.DataFrame.from_dict(team_stats, orient='index')
team_df = team_df.sort_values(['points', 'gd'], ascending=[False, False])
team_rank = {team: rank + 1 for rank, team in enumerate(team_df.index)}
team_strength = {team: (max(team_rank.values()) + 1 - r) for team, r in team_rank.items()}
team_strength_raw = pd.Series(team_strength)
strength_mean = team_strength_raw.mean(); strength_std = team_strength_raw.std() if team_strength_raw.std() > 0 else 1.0
team_strength_z = {t: (v - strength_mean) / strength_std for t, v in team_strength.items()}

# Compute residuals and scale
residuals = []
for _, row in stint_df.iterrows():
    exp = team_strength_z[row['h_team']] - team_strength_z[row['a_team']]
    res = (row['h_goals'] - row['a_goals']) - exp
    residuals.append(res)
residuals = np.array(residuals)
scale = residuals.std() if residuals.std() > 0 else 1.0

# Actual normalized GD for Canada stints
actual_rows = []
for _, row in stint_df.iterrows():
    for side, team_col, opp_col, gf_col, ga_col in [
        ('home', 'h_team', 'a_team', 'h_goals', 'a_goals'),
        ('away', 'a_team', 'h_team', 'a_goals', 'h_goals'),
    ]:
        team_name = row[team_col]
        if team_name != 'Canada':
            continue
        opp = row[opp_col]
        goal_margin = row[gf_col] - row[ga_col]
        expected = team_strength_z[team_name] - team_strength_z[opp]
        norm_gd = (goal_margin - expected) / scale
        actual_rows.append({
            'stint_number': row['stint_number'],
            'game_id': row['game_id'],
            'opponent': opp,
            'actual_norm_gd': norm_gd,
        })
actual_df = pd.DataFrame(actual_rows)
actual_summary = actual_df.groupby('stint_number', as_index=False)['actual_norm_gd'].mean()

# Optimized expected score per stint (using same wowy_factor as optimizer default = 1.0)
wowy_factor = 1.0
opt_rows = []
for stint, data in optimized_lineups.items():
    if data and data['lineup']:
        lineup = data['lineup']
        score = sum(stint_gd[p][stint] + wowy_dict.get(p, 0) * wowy_factor for p in lineup)
        opt_rows.append({'stint_number': stint, 'optimized_score': score})
opt_df = pd.DataFrame(opt_rows)

compare = opt_df.merge(actual_summary, on='stint_number', how='left')
compare['delta_opt_vs_actual'] = compare['optimized_score'] - compare['actual_norm_gd']

print("Comparison of optimized vs actual Canada stints (normalized GD):")
print(compare)
print("\nAverages:")
print(compare[['optimized_score', 'actual_norm_gd', 'delta_opt_vs_actual']].mean())

# Paired t-test vs zero
diffs = compare['delta_opt_vs_actual'].dropna().tolist()
if len(diffs) > 1:
    mean_delta = statistics.mean(diffs)
    std_delta = statistics.pstdev(diffs)
    se = std_delta / math.sqrt(len(diffs)) if std_delta else float('nan')
    t_stat = mean_delta / se if se and se != 0 else float('nan')
else:
    mean_delta = float('nan'); t_stat = float('nan')

print("\nPaired t-stat vs 0 (delta):", t_stat)



Comparison of optimized vs actual Canada stints (normalized GD):
    stint_number  optimized_score  actual_norm_gd  delta_opt_vs_actual
0              1         0.343572       -0.164033             0.507605
1              2         1.000089        0.051520             0.948569
2              3         1.440992       -0.143504             1.584497
3              4         0.546716       -0.179430             0.726145
4              5         1.919477        0.051520             1.867957
5              6         2.298167       -0.053691             2.351857
6              7         1.667842       -0.074219             1.742061
7              8         2.471170       -0.002369             2.473539
8              9         0.832623       -0.017853             0.850476
9             10         1.212186       -0.108139             1.320325
10            11         1.060730        0.012220             1.048510
11            12         0.265571       -0.138345             0.403916
12          